In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_score,
)
from sklearn.base import clone
from scipy.spatial.distance import euclidean
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
from scipy.spatial.distance import cdist
import plotly.graph_objs as go
import seaborn as sns
plt.ioff()
plt.clf()

<Figure size 640x480 with 0 Axes>

In [32]:
from IPython.core.display import HTML

# Apply CSS styling to center all output
HTML("""
<style>
.output_png {
    display: flex;
    justify-content: center;
    align-items: center;
}
</style>
""")

In [3]:
%run 'Lab_preprocess.ipynb'

Validation Criteria

In [4]:
def pooled_within_ssd(X, y, centroids, dist):
    """Compute pooled within-cluster sum of squares around the cluster mean

    Parameters
    ----------
    X : array
        Design matrix with each row corresponding to a point
    y : array
        Class label of each point
    centroids : array
        Cluster centroids
    dist : callable
        Distance between two points. It should accept two arrays, each
        corresponding to the coordinates of each point

    Returns
    -------
    float
        Pooled within-cluster sum of squares around the cluster mean
    """
    total_ssd = 0
    for i, centroid in enumerate(centroids):
        points = X[y == i]
        len_p = len(points)
        ssd = (1/(2*len_p)) * sum(dist(point, centroid)**2 for
                                  point in points)
        total_ssd += ssd
    return total_ssd

In [5]:
def gen_realizations(X, b, random_state=None):
    """Generate b random realizations of X

    The realizations are drawn from a uniform distribution over the range of
    observed values for that feature.

    Parameters
    ---------
    X : array
        Design matrix with each row corresponding to a point
    b : int
        Number of realizations for the reference distribution
    random_state : int, default=None
        Determines random number generation for realizations

    Returns
    -------
    X_realizations : array
        random realizations with shape (b, X.shape[0], X.shape[1])
    """
    mins = X.min(axis=0)
    maxs = X.max(axis=0)
    rng = np.random.default_rng(random_state)
    nrows, ncols = X.shape
    return rng.uniform(
        np.tile(mins, (b, nrows, 1)),
        np.tile(maxs, (b, nrows, 1)),
        size=(b, nrows, ncols),
    )

In [6]:
def gap_statistic(X, y, centroids, dist, b, clusterer, random_state=None):
    """Compute the gap statistic

    Parameters
    ----------
    X : array
        Design matrix with each row corresponding to a data point
    y : array
        Class label of each point
    centroids : array
        Cluster centroids
    dist : callable
        Distance between two points. It should accept two arrays, each
        corresponding to the coordinates of each point
    b : int
        Number of realizations for the reference distribution
    clusterer : KMeans
        Clusterer object that will be used for clustering the reference
        realizations
    random_state : int, default=None
        Determines random number generation for realizations

    Returns
    -------
    gs : float
        Gap statistic
    gs_std : float
        Standard deviation of gap statistic
    """
    X_refs = gen_realizations(X, b, random_state)
    pooled_ssd = pooled_within_ssd(X, y, centroids, dist)
    log_ssd = np.log(pooled_ssd)
    log_ref = np.zeros(b)

    for i in range(5):
        clusterer.fit(X_refs[i])
        centroids_ref = clusterer.cluster_centers_
        labels_ref = clusterer.labels_
        ssd_ref = pooled_within_ssd(X_refs[i], labels_ref,
                                    centroids_ref, dist)
        log_ref[i] = np.log(ssd_ref)

    gap_stat = log_ref.mean() - log_ssd
    std_dev = np.std(log_ref)
    return ([gap_stat, std_dev])

In [7]:
def cluster_range(X, clusterer, k_start, k_stop):
    """Cluster X for different values of k

    Parameters
    ----------
    X : array
        Design matrix with each row corresponding to a data point
    clusterer : sklearn.base.ClusterMixin
        Perform clustering for different value of `k` using this model. It
        should have been initialized with the desired parameters
    k_start : int
        Perform k-means starting from this value of `k`
    k_stop : int
        Perform k-means up to this value of `k` (inclusive)

    Returns
    -------
    dict
        The value of each key is a list with elements corresponding to the
        value at `k`. The keys are:
            * `ys`: cluster labels
            * `centers`: cluster centroids
            * `inertias`: sum of squared distances to the cluster centroid 
            * `chs`: Calinski-Harabasz indices
            * `scs`: silhouette coefficients
            * `dbs`: Davis-Bouldin indices
            * `gss`: gap statistics
            * `gssds`: standard deviations of gap statistics
    """
    returns = {
        'ys': [],
        'centers': [],
        'inertias': [],
        'chs': [],
        'scs': [],
        'dbs': [],
        'gss': [],
        'gssds': []}
    for k in range(k_start, k_stop + 1):
        clusterer_k = clone(clusterer).set_params(
            n_clusters=k, random_state=1337)
        label = clusterer_k.fit_predict(X)
        gs = gap_statistic(
            X,
            label,
            clusterer_k.cluster_centers_,
            euclidean,
            5,
            clone(clusterer).set_params(n_clusters=k),
            random_state=1337,
        )

        returns['ys'].append(label)
        returns['centers'].append(clusterer_k.cluster_centers_)
        returns['inertias'].append(clusterer_k.inertia_)
        returns['chs'].append(calinski_harabasz_score(X, label))
        returns['scs'].append(silhouette_score(X, label) if k > 1 else None)
        returns['dbs'].append(davies_bouldin_score(
            X, label) if k > 1 else None)
        returns['gss'].append(gs[0])
        returns['gssds'].append(gs[1])
    return returns

In [13]:
def plot_internal(inertias, chs, scs, dbs, gss, gssds):
    """Plot internal validation values"""
    fig, ax = plt.subplots(figsize=(10, 6))
    ks = np.arange(2, len(inertias) + 2)

    # Assigning colors
    colors = ['#205e55', '#8e9b8c', '#7cb79d', '#5a8a7b', '#b1cbb5']

    # Plotting with the new color scheme
    ax.plot(ks, inertias, "-o", label="SSE", color=colors[0])
    ax.plot(ks, chs, "-o", label="CH", color=colors[1])
    ax.set_xlabel("$k$")
    ax.set_ylabel("SSE/CH")

    ax2 = ax.twinx()
    ax2.errorbar(ks, gss, yerr=gssds, fmt="-o", label="GS",
                 color=colors[2])
    ax2.plot(ks, scs, "-o", label="SC", color=colors[3])
    ax2.plot(ks, dbs, "-o", label="DB", color=colors[4])
    ax2.set_ylabel("Gap statistic/Silhouette/DB")

    # Handling legends from both axes
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper left',
               bbox_to_anchor=(1.05, 1))

    # Adding a title
    ax.set_title("Internal Validation Metrics for K-Medoids")
    plt.tight_layout()

    return fig

In [11]:
country_med = cluster_range(
    country_pca.to_numpy(), KMedoids(random_state=1337, method='pam'), 2, 15
)

In [17]:
medoids_plot_internal = plot_internal(
    country_med["inertias"],
    country_med["chs"],
    country_med["scs"],
    country_med["dbs"],
    country_med["gss"],
    country_med["gssds"],
);

In [ ]:
def plotting(x, y, z, cluster_centers, labels):
    """
    Generate a 3D scatter plot of data points with custom color mapping.

    Parameters
    ----------
    x : array_like
        The x-coordinates of the data points.
    y : array_like
        The y-coordinates of the data points.
    z : array_like
        The z-coordinates of the data points.
    labels : array_like
        Cluster labels for each data point.

    Returns
    -------
    Figure
        A Plotly Figure object containing the 3D scatter plot.

    """
    x = x
    y = y
    z = z

    custom_colors = ['#205e55', '#8e9b8c', '#7cb79d']

    # Create a list of colors for each label
    color_mapped = [custom_colors[label] for label in labels]

    trace_data = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=8,
            color=color_mapped,  # set color to an array/list of desired values
            colorscale='Viridis',  # choose a colorscale
            opacity=0.8
        )
    )

    # Adding trace for cluster centers
    trace_centers = go.Scatter3d(
        x=cluster_centers[:, 0],
        y=cluster_centers[:, 1],
        z=cluster_centers[:, 2],
        mode='markers',
        marker=dict(
            size=5,
            color='red',  # color based on cluster index
            colorscale='Viridis',  # choose a colorscale
            opacity=0.8,
            symbol='square',  # use a square marker
            line=dict(color='black', width=2)  # black border around markers
        )
    )

    data = [trace_data, trace_centers]

    layout = go.Layout(
        margin=dict(l=0, r=0, b=0, t=0),  # tight layout
        title='K-Medoids: Scatter Plot',
        scene=dict(
            xaxis=dict(title='PCA 1'),
            yaxis=dict(title='PCA 2'),
            zaxis=dict(title='PCA 3')
        )
    )

    fig = go.Figure(data=data, layout=layout)

    # Render the plot
    return fig

## K-Medoids

In [19]:
import matplotlib.pyplot as plt

medoids_country = KMedoids(n_clusters=3, random_state=1337, method='pam')
medoids_country.fit(country_pca)
y_predict_country_medoids = medoids_country.fit_predict(country_pca)
cluster_centers_medoids = medoids_country.cluster_centers_


def medoids_scatter():
    """
    Generate a scatter plot of clustered data with custom color mapping.

    Returns
    -------
    Figure
        A matplotlib Figure object containing the scatter plot.

    """
    custom_colors = ['#205e55', '#8e9b8c', '#7cb79d']

    # Create a list of colors for each label
    color_map = {i: custom_colors[i] for i
                 in range(medoids_country.n_clusters)}
    color_mapped = [custom_colors[label] for label
                    in y_predict_country_medoids]
    cluster_colors = [color_map[i] for i in range(medoids_country.n_clusters)]

    # Setting the figure size
    fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the size as needed

    plt.scatter(country_pca.iloc[:, 0], country_pca.iloc[:, 1], c=color_mapped)
    plt.scatter(
        cluster_centers_medoids[:, 1],
        cluster_centers_medoids[:, 2],
        s=60,
        c=cluster_colors,
        marker="s",
        ec="k",
        lw=2,
    )

    plt.tight_layout()  # Adjust the layout
    return fig

In [20]:
medscatter = medoids_scatter();

In [19]:
medoids_3d = plotting(country_pca.iloc[:, 0], country_pca.iloc[:, 1],
                      country_pca.iloc[:, 2], cluster_centers_medoids, y_predict_country_medoids)

In [35]:
# Calculate distances from each centroid to each point in the PCA dataset
distances = cdist(cluster_centers_medoids, country_pca)

# Find the index of the minimum distance for each centroid
centroid_indices = np.argmin(distances, axis=1)

# Index 63 is the centroid for label 0, 46 for 1, and 39 for 

In [36]:
selected_rows = df_final.iloc[centroid_indices].iloc[:, 2:]

In [28]:

def centroids_plot():
    """
    Generate a line plot for centroid features.

    This function creates a line plot to visualize the features of centroids
    obtained from a clustering process. Each centroid's features are plotted as 
    a separate line with unique colors and labels. Assumes the existence of 
    'selected_rows' and 'df_final' data structures.

    Returns
    -------
    Figure
        A matplotlib Figure object containing the centroids plot.

    """
    fig, ax = plt.subplots(figsize=(15, 6))  # Adjust the size as needed
    custom_colors = ['#205e55', '#8e9b8c', '#7cb79d']
    # Get column names for the x-axis
    columns = selected_rows.columns

    # Plot each row
    for index, row in enumerate(selected_rows.iterrows()):
        color = custom_colors[index % len(custom_colors)]
        label = f'{df_final.iloc[row[0]].iloc[0]}'
        ax.plot(columns, row[1], marker='o', color=color, label=label)

    # Customize the plot
    ax.set_xticks(columns)  # Set the x-ticks to be the column names
    ax.set_xticklabels(columns, rotation=90)
    ax.set_ylabel('Value')
    ax.set_title('K-Medoids: Centroids Defining Features')
    ax.legend()

    # Show the plot
    plt.tight_layout()
    return fig

In [23]:
df_final['label'] = y_predict_country_medoids
df_graph = df_final.iloc[:, 2:]

In [24]:
# Compute the min and max for each column grouped by the label
grouped = df_graph.groupby('label')
min_values = grouped.min()
max_values = grouped.max()
mean_values = grouped.mean()

# Plot setup for 13 rows and 2 columns, adjust figsize as necessary


def medoids_range_plot():
    """
    Generate range plots for different features by labels.

    Returns
    -------
    Figure
        A matplotlib Figure object with the range plots.

    """
    fig, axes = plt.subplots(5, 5, figsize=(15, 2 * 5))
    axes = axes.flatten()  # Flatten the array of axes

    labels = [0, 1, 2]
    colors = ['#205e55', '#8e9b8c', '#7cb79d']

    # Loop through each feature column to create a range bar
    for i, col in enumerate(df_graph.columns[:-1]):  # Exclude the label column
        ax = axes[i]
        bar_height = 0.5  # The height of the bars
        for j, label in enumerate(labels):
            # Plotting the range as a horizontal bar
            ax.barh(
                y=label,
                width=max_values[col][label] - min_values[col][label],
                left=min_values[col][label],
                height=bar_height,
                color=colors[j],
                edgecolor='black',
                label=f'Label {label}' if i % (len(labels) * 2) == 0 else ""
            )
            # Plotting the mean as a short horizontal line, inside the bar
            mean_value = mean_values[col][label]
            ax.plot(
                [mean_value, mean_value],  # X start and end of the line
                [label - bar_height / 2, label + bar_height / 2],
                color='black',  # Color of the mean line
                linestyle='--',  # Style of the line
                linewidth=2,  # Width of the line
                label=f'Mean for Label {label}' if i == 0 else ""
            )

        ax.set_title(f'Range for {col}')
        ax.set_yticks(labels)
        ax.set_yticklabels(labels)
        ax.set_ylim(min(labels) - bar_height, max(labels) + bar_height)

    plt.tight_layout()  # Adjust the layout
    return fig

In [26]:
med_range = medoids_range_plot();

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns


def medoids_box():
    """
    Generate box plots for different features by labels.

    Returns
    -------
    Figure
        A matplotlib Figure object with the range plots.

    """
    # Set up the matplotlib figure with 13 rows and 2 columns
    fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(15, 10))

    axes = axes.flatten()  # Flatten the 2D array of axes for easy iteration

    # Assuming 'df_graph' is your DataFrame and the last column is 'label'
    feature_columns = df_graph.columns[:-1]  # Exclude the label column
    colors = ['#205e55', '#8e9b8c', '#7cb79d']
    # Loop through each feature column to create a box plot
    for i, col in enumerate(feature_columns):
        sns.boxplot(x='label', y=col, data=df_graph, ax=axes[i],
                    palette=colors,
                    hue='label', legend=False)
        axes[i].set_title(f'Values for {col}')
        axes[i].set_xlabel('Label')
        axes[i].set_ylabel('Value')

    # Adjust the layout
    plt.tight_layout()

    # Display the plot
    return fig